In [1]:
import os 

root_dir = os.path.join("/work/hzeng_umass_edu/ir-research/joint_modeling_search_and_rec/datasets/unified_kgc/unified_user/")

static_dir = os.path.join(root_dir, "without_context")
seq_dir = os.path.join(root_dir, "sequential_train_test")

In [2]:
def from_qrel_to_u2p(urel_path):
    uid_to_pos_pids = {}
    with open(urel_path) as fin:
        print(urel_path)
        for line in fin:
            uid, _, pos_pid, _ = line.strip().split("\t")
            uid, pos_pid = int(uid), int(pos_pid)
            if uid not in uid_to_pos_pids:
                uid_to_pos_pids[uid] = [pos_pid]
            else:
                uid_to_pos_pids[uid] = [pos_pid]
    return uid_to_pos_pids

static_search_path = os.path.join(static_dir, "urels.test.search.tsv")
seq_search_path = os.path.join(seq_dir, "urels.search.test.tsv")

static_search_u2p = from_qrel_to_u2p(static_search_path)
seq_search_u2p = from_qrel_to_u2p(seq_search_path)

FileNotFoundError: [Errno 2] No such file or directory: '/work/hzeng_umass_edu/ir-research/joint_modeling_search_and_rec/datasets/unified_kgc/unified_user/without_context/urels.test.search.tsv'

In [ ]:
#uids = list(static_search_u2p.keys())[:10]
for uid in static_search_u2p.keys():
    static_pids = static_search_u2p[uid]
    seq_pids = seq_search_u2p[uid]
    
    assert set(static_pids) == set(seq_pids)
    
    

In [22]:
import ujson

train_path="/home/jupyter/unity_jointly_rec_and_search/datasets/unified_user/sequential_train_test/hlen_4_bm25/search_sequential.train.json"
test_path="/home/jupyter/unity_jointly_rec_and_search/datasets/unified_user/sequential_train_test/hlen_4_bm25/search_sequential.test.json"
compl_train_path = "/home/jupyter/unity_jointly_rec_and_search/datasets/unified_user/sequential_train_test/hlen_4_bm25/compl_rec_sequential.train.json"


train_examples = []
test_examples = []
compl_train_examples = []

with open(train_path) as fin:
    for line in fin:
        train_examples.append(ujson.loads(line))
        
with open(test_path) as fin:
    for line in fin:
        test_examples.append(ujson.loads(line))
        
with open(compl_train_path) as fin:
    for line in fin:
        compl_train_examples.append(ujson.loads(line))

In [ ]:
test_uids = set([example["uid"] for example in test_examples])
train_uids = set([example["uid"] for example in train_examples])
compl_train_uids = set([example["uid"] for example in compl_train_examples])
train_uid_to_queryids = {x["uid"]: x["query_ids"] for x in train_examples if x["uid"] in test_uids}
test_uid_to_queryids = {x["uid"]: x["query_ids"] for x in test_examples if x["uid"]}
train_uid_to_valueids = {x["uid"]: x["context_value_ids"] for x in train_examples if x["uid"] in test_uids}
test_uid_to_valueids = {x["uid"]: x["context_value_ids"] for x in test_examples if x["uid"]}
print(len(train_uid_to_queryids), len(test_uid_to_queryids))
for train_queryids, test_queryids in zip(train_uid_to_queryids.values(), test_uid_to_queryids.values()):
    assert train_queryids == test_queryids[:-1]
for train_valueids, test_valueids in zip(train_uid_to_valueids.values(), test_uid_to_valueids.values()):
    assert train_valueids == test_valueids[:-1]
print(test_examples[:2])
print("overlap users between train search and compl = {:,}".format(len(train_uids & compl_train_uids)))
print("number of train search and compl = {:,}, {:,}".format(len(train_uids), len(compl_train_uids)))

In [12]:
root_dir="/home/jupyter/unity_jointly_rec_and_search/datasets/unified_user/"
eid_to_text = {}
with open(os.path.join(root_dir, "all_entities.tsv")) as fin:
    for line in fin:
        eid, text = line.strip().split("\t")
        eid_to_text[int(eid)] = text

In [9]:
rels_path="/home/jupyter/unity_jointly_rec_and_search/datasets/unified_user/sequential_train_test/urels.search.test.tsv"
! head -n 5 $rels_path

148	Q0	1534880	1
182	Q0	1019276	1
416	Q0	1787563	1
580	Q0	2205787	1
581	Q0	2134319	1


In [16]:
import os
import pickle
import glob

import pandas as pd 
import numpy as np

bq_in_dir="/home/jupyter/data_transfer/data/"
with open(os.path.join(bq_in_dir, "time_anchor_5core_compl_rec_bytime.pkl"), "rb") as fin:
    user_compl_rec_df = pickle.load(fin)

In [17]:
user_compl_rec_df.head()

,customer_id,compl_records
0,1824001604224154582|3162928508271163957,"[{'date_time': '2021-12-31 20:37:38', 'anchor'..."
1,4654957148783008295|8571441076751289752,"[{'date_time': '2022-02-01 20:36:51', 'anchor'..."
2,8299603424924873952|1058200923248039745,"[{'date_time': '2022-04-03 14:25:13', 'anchor'..."
3,3475932232069669088|8896859845948872617,"[{'date_time': '2021-12-17 20:12:11', 'anchor'..."
4,5638287371680201640|3535992140736785982,"[{'date_time': '2022-06-12 22:56:08', 'anchor'..."
